In [1]:
import pandas as pd
data=pd.read_csv('D:\\PGP-BABI\\Capstone\\Airbnb\\Tasmania-detailed reviews.csv', error_bad_lines=False)
#Drop rows with blank comments
data_valid = data.dropna()
#Group all comments for a single listing
data_grouped = data_valid.groupby('listing_id', sort=False)['comments'].apply(' '.join).reset_index()

In [1]:
data.loc[0:2,"comments"]

NameError: name 'data' is not defined

In [2]:
data_grouped

,listing_id,comments
0,35644,Yvette picked us up from the bus stop on our a...
1,38951,Sue was a great host and very thoughtful. Sand...
2,84581,This home is like a little bit of Tuscany in T...
3,88873,We would recommend this accommodation for anyo...
4,137808,"We had a family wedding near Launceston, Tasma..."
5,162081,We could not be happier! Pat and John have a ...
6,252996,Let me just say that Kit was the best host you...
7,268482,The host canceled this reservation 105 days be...
8,270374,Kits place was absolutely exquisite. It was ab...
9,278045,Jesse and Katie are great hosts. Their house ...


In [2]:
cluster_words=pd.read_csv('D:\\PGP-BABI\\Capstone\\Airbnb\\Clustering\\review\\review_cluster_CBOW_w3_spaCy_mincount_1000_10 - Upd.csv', error_bad_lines=False)
cluster_words[:2]

,word,cluster,Unnamed: 2
0,bus,4,NaN
1,arrival,5,NaN


In [3]:
#Create a list of distinct cluster values to loop through
cluster_values=list(set(cluster_words['cluster']))
cluster_values

[0, 1, 2, 3, 4, 5, 6, 7]

In [1]:
#Import libraries
import datetime
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
#import nltk
#nltk.download('punkt') #This is one time
import spacy
#import en_core_web_sm
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

#conda install rpy2 #Run in cmd prompt
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
utils = importr('utils')
#utils.install_packages('syuzhet') #This is one time
ro.r('library(syuzhet)')

#parallel processing
import numpy as np
import seaborn as sns
from multiprocessing import Pool

In [71]:
#Test syuzhet sentiment analyzer
print(ro.r('get_sentiment("This is filthy.")'))
print(ro.r('get_nrc_sentiment("This is filthy.")'))
print(ro.r('get_nrc_sentiment("This is filthy.")$negative'))
nrc_test=ro.r('get_nrc_sentiment("This is filthy.")')
print(nrc_test[8])

[1] -0.5

  anger anticipation disgust fear joy sadness surprise trust negative positive

1     0            0       1    0   0       0        0     0        1        0

[1] 1

[1] 1



In [5]:
#Create function to map sentences to features and assign sentiment score per feature
def map_sentence(row):
    result = pd.DataFrame([])
    text=row['comments']
    text=repr(text) # Remove non-unicode characters
    text=((text.encode('ascii','ignore')).decode("utf-8")).lower()
    
    # Remove new line characters
    text = text.replace("\\r\\n", " ")
    #text = text.replace("\\n\\n", " ")
    text = text.replace("\\n", " ")
    
    #Sentence tokenization
    for token in sent_tokenize(text):
        sent_cluster=[]
        temp=[]

        #Word tokenization
        tokenized_words=word_tokenize(token)

        #Lemmatization and POS tagging - Extract only Nouns and Adjectives
        allowed_postags=['NOUN', 'ADJ']
        doc = nlp(" ".join(tokenized_words)) 
        #for doc in nlp.pipe(" ".join(tokenized_words), batch_size=10000, n_threads=3):
            #pass
        temp.append([t.lemma_ for t in doc if t.pos_ in allowed_postags])
        temp=temp[0]

        #Sentence-to-feature one hot encoding
        for i in cluster_values:
            z=0
            for word in temp:
                if word in list(cluster_words.loc[cluster_words['cluster']==i]['word']):
                    z=z+1
            if z>0:
                sent_cluster.append(i)

        #Sentiment score using Vader
        score = analyser.polarity_scores(token)

        #Create data frame
        #df = pd.DataFrame({'listing_id':row['listing_id'],'sentence':token,'features':[sent_cluster],'pos_score':score['pos'],'neg_score':score['neg'],'neu_score':score['neu'],'compound_score':score['compound']})
        ro.globalenv["token"] = token
        df = pd.DataFrame({'listing_id':row['listing_id'],'sentence':token,'features':[sent_cluster],'score_nrc_pos':ro.r('get_nrc_sentiment(token)')[9],'score_nrc_neg':ro.r('get_nrc_sentiment(token)')[8],'score_overall':ro.r('get_sentiment(token)'),'pos_score_vader':score['pos'],'neg_score_vader':score['neg'],'neu_score_vader':score['neu'],'compound_score_vader':score['compound']})
        result = result.append(df)
        
    return result

#TODO: Avg sentiment score per feature + avg overall sentiment score, for each listing

In [200]:
#Test with vader only
test=data_grouped.iloc[2]
print(data_grouped['comments'][2])
print(map_sentence(test))

This home is like a little bit of Tuscany in Tasmania!

We spent my husbands 50th here with our family and so glad we did.

Melissa was a most welcoming and considerate host. We were greeted warmly and given a tour to ensure we were comfortable with everything. The house is very private with beautiful vistas. 

We were in striking distance of Sheffield, Stanley and Cradle Mountain and found ourselves walking most days. The drive to Cradle Mountain was easy and very scenic and built the anticipation and appreciation of the beauty of this place on the way

We spent beautiful evenings outside playing cards and chatting.

This home seems to bring joy with it.

Thank you Melissa.

 在这里度过了完美的两天~这里是一栋装修复古豪华又精致的两层楼独栋｡处处体现女主人的品味和细心,全家都赞不绝口!女主人非常非常nice｡以后还有机会一定再去!路到头有一个农场养了一匹偏分马~哈哈哈!
We love it!Thank you Melissa! Hi, We stayed 1 night in Melissa's lovely home. Melissa went out of her way to make us feel welcome, including making me a beautiful home made birthday cake. 
Home has amazing view and 

0              [1]      0.000      0.000      1.000          0.0000  


In [30]:
#Test with get_sentiment only
test=data_grouped.iloc[2]
print(data_grouped['comments'][2])
print(map_sentence(test))

This home is like a little bit of Tuscany in Tasmania!

We spent my husbands 50th here with our family and so glad we did.

Melissa was a most welcoming and considerate host. We were greeted warmly and given a tour to ensure we were comfortable with everything. The house is very private with beautiful vistas. 

We were in striking distance of Sheffield, Stanley and Cradle Mountain and found ourselves walking most days. The drive to Cradle Mountain was easy and very scenic and built the anticipation and appreciation of the beauty of this place on the way

We spent beautiful evenings outside playing cards and chatting.

This home seems to bring joy with it.

Thank you Melissa.

 在这里度过了完美的两天~这里是一栋装修复古豪华又精致的两层楼独栋｡处处体现女主人的品味和细心,全家都赞不绝口!女主人非常非常nice｡以后还有机会一定再去!路到头有一个农场养了一匹偏分马~哈哈哈!
We love it!Thank you Melissa! Hi, We stayed 1 night in Melissa's lovely home. Melissa went out of her way to make us feel welcome, including making me a beautiful home made birthday cake. 
Home has amazing view and 

0              [1]             0.50  


In [73]:
#Test with vader, get_sentiment, get_nrc_sentiment
test=data_grouped.iloc[2]
print(data_grouped['comments'][2])
print(map_sentence(test))

This home is like a little bit of Tuscany in Tasmania!

We spent my husbands 50th here with our family and so glad we did.

Melissa was a most welcoming and considerate host. We were greeted warmly and given a tour to ensure we were comfortable with everything. The house is very private with beautiful vistas. 

We were in striking distance of Sheffield, Stanley and Cradle Mountain and found ourselves walking most days. The drive to Cradle Mountain was easy and very scenic and built the anticipation and appreciation of the beauty of this place on the way

We spent beautiful evenings outside playing cards and chatting.

This home seems to bring joy with it.

Thank you Melissa.

 在这里度过了完美的两天~这里是一栋装修复古豪华又精致的两层楼独栋｡处处体现女主人的品味和细心,全家都赞不绝口!女主人非常非常nice｡以后还有机会一定再去!路到头有一个农场养了一匹偏分马~哈哈哈!
We love it!Thank you Melissa! Hi, We stayed 1 night in Melissa's lovely home. Melissa went out of her way to make us feel welcome, including making me a beautiful home made birthday cake. 
Home has amazing view and 

0            0.000            0.000            1.000                0.0000  


In [12]:
#Save test results
map_sentence(test).to_csv('D:\\PGP-BABI\\Capstone\\Airbnb\\Sentiment Scores\\test_listing_multiple.csv', sep=',')

ValueError: array length 1 does not match index length 50

In [124]:
#Test for a bad review
test=data_grouped.loc[data_grouped['listing_id']==17085056]
print(test['comments'])
print(map_sentence(test))
#Note: both vader and syuzhet give neutral score

2173    The host canceled this reservation 21 days bef...
Name: comments, dtype: object
      listing_id                                           sentence features  \
2173    17085056  2173    the host canceled this reservation 21 ...   [0, 1]   

      score_nrc_pos  score_nrc_neg  score_overall  pos_score_vader  \
2173            0.0            0.0            0.0              0.0   

      neg_score_vader  neu_score_vader  compound_score_vader  
2173              0.0              1.0                   0.0  


In [83]:
test="Place was filthy and you stay in a room but there are permanent boarders to share amenities with. No sheets on the bed, just a doona, no blankets and had to ask for toilet paper and then provide my own. Had no interaction with the host other than a collection from the airport and the noise from the street and the walls being paper thin. I could go on but this was my first experience using AIRBNB so it is with great hesitation that I would use it again based on this experience."
print(test)
for token in sent_tokenize(test):
    ro.globalenv["token"] = token
    print(token)
    print(ro.r('get_nrc_sentiment(token)'))
    print(ro.r('get_sentiment(token)'))
    print(analyser.polarity_scores(token))

Place was filthy and you stay in a room but there are permanent boarders to share amenities with. No sheets on the bed, just a doona, no blankets and had to ask for toilet paper and then provide my own. Had no interaction with the host other than a collection from the airport and the noise from the street and the walls being paper thin. I could go on but this was my first experience using AIRBNB so it is with great hesitation that I would use it again based on this experience.
Place was filthy and you stay in a room but there are permanent boarders to share amenities with.
  anger anticipation disgust fear joy sadness surprise trust negative positive

1     0            1       1    0   1       0        0     1        1        1

[1] -0.25

{'neg': 0.0, 'neu': 0.851, 'pos': 0.149, 'compound': 0.4215}
No sheets on the bed, just a doona, no blankets and had to ask for toilet paper and then provide my own.
  anger anticipation disgust fear joy sadness surprise trust negative positive

1  

In [85]:
#Another test
test = "the pictures don't do it justice, you really need to see it in person."
print(test)
for token in sent_tokenize(test):
    ro.globalenv["token"] = token
    print(token)
    print(ro.r('get_nrc_sentiment(token)'))
    print(ro.r('get_sentiment(token)'))
    print(analyser.polarity_scores(token))
#Note: syuzhet gives positive result, which is correct, while vader gives a negative result, which is incorrect

the pictures don't do it justice, you really need to see it in person.
the pictures don't do it justice, you really need to see it in person.
  anger anticipation disgust fear joy sadness surprise trust negative positive

1     0            0       0    0   0       0        0     1        0        1

[1] 1

{'neg': 0.176, 'neu': 0.824, 'pos': 0.0, 'compound': -0.4168}


In [265]:
#Extras
#Testing updates to lexicon
new_words = {
    'foo': 2.0,
    'bar': -3.4,
}
SIA = SentimentIntensityAnalyzer()
SIA.lexicon.update(new_words)
SIA.lexicon.pop('bar')
SIA.lexicon
#type(SIA.lexicon)

{'$:': -1.5,
 '%)': -0.4,
 '%-)': -1.5,
 '&-:': -0.4,
 '&:': -0.7,
 "( '}{' )": 1.6,
 '(%': -0.9,
 "('-:": 2.2,
 "(':": 2.3,
 '((-:': 2.1,
 '(*': 1.1,
 '(-%': -0.7,
 '(-*': 1.3,
 '(-:': 1.6,
 '(-:0': 2.8,
 '(-:<': -0.4,
 '(-:o': 1.5,
 '(-:O': 1.5,
 '(-:{': -0.1,
 '(-:|>*': 1.9,
 '(-;': 1.3,
 '(-;|': 2.1,
 '(8': 2.6,
 '(:': 2.2,
 '(:0': 2.4,
 '(:<': -0.2,
 '(:o': 2.5,
 '(:O': 2.5,
 '(;': 1.1,
 '(;<': 0.3,
 '(=': 2.2,
 '(?:': 2.1,
 '(^:': 1.5,
 '(^;': 1.5,
 '(^;0': 2.0,
 '(^;o': 1.9,
 '(o:': 1.6,
 ")':": -2.0,
 ")-':": -2.1,
 ')-:': -2.1,
 ')-:<': -2.2,
 ')-:{': -2.1,
 '):': -1.8,
 '):<': -1.9,
 '):{': -2.3,
 ');<': -2.6,
 '*)': 0.6,
 '*-)': 0.3,
 '*-:': 2.1,
 '*-;': 2.4,
 '*:': 1.9,
 '*<|:-)': 1.6,
 '*\\0/*': 2.3,
 '*^:': 1.6,
 ',-:': 1.2,
 "---'-;-{@": 2.3,
 '--<--<@': 2.2,
 '.-:': -1.2,
 '..###-:': -1.7,
 '..###:': -1.9,
 '/-:': -1.3,
 '/:': -1.3,
 '/:<': -1.4,
 '/=': -0.9,
 '/^:': -1.0,
 '/o:': -1.4,
 '0-8': 0.1,
 '0-|': -1.2,
 '0:)': 1.9,
 '0:-)': 1.4,
 '0:-3': 1.5,
 '0:03': 1.9,
 '

In [ ]:
len(SIA.lexicon)

In [272]:
print(analyser.polarity_scores("do not"))
print(analyser.polarity_scores("don't"))
print(analyser.polarity_scores("filthy"))
#Note: All the above words are currently marked as neutral as they don't exist in the default Vader lexicon

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [5]:
#Extras
#Import external nrc lexicon dictionary
import csv
nrc_lexicon_list = []
with open('D:\\PGP-BABI\\Capstone\\Airbnb\\Sentiment Scores\\NRC-Sentiment-Emotion-Lexicons\\NRC-Sentiment-Emotion-Lexicons\\NRC-Emotion-Lexicon-v0.92\\NRC-Emotion-Lexicon-Wordlevel-v0.92.txt', newline = '') as nrc_lexicon:
    lexicon = csv.reader(nrc_lexicon, delimiter='\t')
    for line in lexicon:
        nrc_lexicon_list.append(line)

In [316]:
#Extras
#Extract only positive and negative scores
nrc_lexicon_final = pd.DataFrame([])
for line in nrc_lexicon_list[1:20]:
    if line[1]=='negative':
        df = pd.DataFrame({'word':[line[0]],'pos_neg':[line[1]],'score':[line[2]]}) #Does not work
        nrc_lexicon_final.append(df)

In [317]:
nrc_lexicon_final

""


In [10]:
#Create a feature-sentiment-dataframe for all listings
test=data_grouped.iloc[0:50]
fea_sent_df = pd.DataFrame([])
print(datetime.datetime.now())
for index,row in test.iterrows():
#for index,row in data_grouped.iterrows():
    fea_sent_df = fea_sent_df.append(pd.DataFrame(map_sentence(row)))
print(datetime.datetime.now())
fea_sent_df.to_csv('D:\\PGP-BABI\\Capstone\\Airbnb\\Sentiment Scores\\50_listing_feature_sentiment-Upd.csv', sep=',')

2019-05-18 17:43:08.650560
2019-05-18 18:06:13.608290


In [ ]:
#Parallel processing
num_partitions = 10 #number of partitions to split dataframe
num_cores = 2 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

#Create a feature-sentiment-dataframe for all listings
test=data_grouped.iloc[0:50]
fea_sent_df = pd.DataFrame([])
print(datetime.datetime.now())
fea_sent_df = parallelize_dataframe(test, map_sentence)
print(datetime.datetime.now())
fea_sent_df.to_csv('D:\\PGP-BABI\\Capstone\\Airbnb\\Sentiment Scores\\50_listing_feature_sentiment-Upd1.csv', sep=',')


2019-05-18 18:50:30.899598
